In [174]:
import pandas as pd
import numpy as np

In [175]:
fist= pd.read_csv('fist_10s_test_emg.csv')
gun = pd.read_csv('gun_10s_test_emg.csv')
half_heart = pd.read_csv('half_heart_10s_test_emg.csv')
paper = pd.read_csv('paper_10s_test_emg.csv')
zero = pd.read_csv('zero_10s_test_emg.csv')
unknown = pd.read_csv('unknown_10s_test_emg.csv')


In [176]:
#Data Normalization & concatenation

fist_normalized = (fist - fist.min()) / (fist.max() - fist.min())
fist_normalized.head()
fist_normalized['y']=0
fist_normalized.tail()

gun_normalized = (gun - gun.min()) / (gun.max() - gun.min())
gun_normalized.head()
gun_normalized['y']=1
gun_normalized.tail()

half_heart_normalized = (half_heart - half_heart.min()) / (half_heart.max() - half_heart.min())
half_heart_normalized.head()
half_heart_normalized['y']= 2
half_heart_normalized.tail()

paper_normalized = (paper - paper.min()) / (paper.max() - paper.min())
paper_normalized.head()
paper_normalized['y']=3
paper_normalized.tail()

zero_normalized = (zero - zero.min()) / (zero.max() - zero.min())
zero_normalized.head()
zero_normalized['y']=4
zero_normalized.tail()

unknown_normalized = (unknown - unknown.min()) / (unknown.max() - unknown.min())
unknown_normalized.head()
unknown_normalized['y']=5
unknown_normalized.tail()


df1=pd.concat([fist_normalized,gun_normalized],axis= 0)
df1.index= range(1,len(fist_normalized)+len(gun_normalized)+1)
df1.head()

df2=pd.concat([df1,half_heart_normalized],axis= 0)
df2.index=  range(1,len(df1)+len(half_heart_normalized)+1)
df2.head()

df3=pd.concat([df2,paper_normalized],axis= 0)
df3.index=  range(1,len(df2)+len(paper_normalized)+1)
df3.tail()

df4=pd.concat([df3,zero_normalized],axis= 0)
df4.index=  range(1,len(df3)+len(zero_normalized)+1)
df4.tail()

df=pd.concat([df4,unknown_normalized],axis= 0)
df.index=  range(1,len(df4)+len(unknown_normalized)+1)
df.tail()

df['y'].unique()


array([0, 1, 2, 3, 4, 5])

In [180]:
# Defining training and testing sets plus standard scaling

x=df[['Channel_1','Channel_2','Channel_3','Channel_4','Channel_5','Channel_6','Channel_7','Channel_8']]
y=df['y']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25, random_state=42)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = pd.DataFrame(sc.fit_transform(x_train))
x_test = pd.DataFrame(sc.transform(x_test))


In [183]:
# Random forest classifier plus grid search & predictions

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

lr_grid = {'max_depth' : [4,8,16,32,64,128],
           'criterion' : ['entropy','gini']}

clf = RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=42)

gs = GridSearchCV(estimator = clf, param_grid=lr_grid,cv = 5)
gs.fit(x_train,y_train)
y_pred = gs.predict(x_test)

In [168]:
# Metrics

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print('Classification Report: \n', classification_report(y_test,y_pred))
print('Confusion Matrix: \n', confusion_matrix(y_test,y_pred))

Classification Report: 
               precision    recall  f1-score   support

           0       0.95      0.96      0.96       139
           1       0.93      0.90      0.91       125
           2       0.80      0.87      0.83       115
           3       0.92      0.92      0.92       121
           4       0.94      0.90      0.92       129
           5       0.98      0.97      0.97       118

    accuracy                           0.92       747
   macro avg       0.92      0.92      0.92       747
weighted avg       0.92      0.92      0.92       747

Confusion Matrix: 
 [[134   1   2   2   0   0]
 [  0 112   7   4   1   1]
 [  4   4 100   1   5   1]
 [  1   0   8 111   1   0]
 [  1   1   8   3 116   0]
 [  1   3   0   0   0 114]]


In [173]:
print("Best Parameters: ", gs.best_params_)

Best Parameters:  {'criterion': 'gini', 'max_depth': 32}
